In [1]:
# Based on those articles/blog:
# pytoch implementation of cgn: https://arxiv.org/pdf/1609.02907.pdf
# http://tkipf.github.io/graph-convolutional-networks/

In [2]:
import torch
import numpy as np
from torch.autograd import Variable
import torch.nn.functional as F
from torch import nn
import time

In [3]:
# Create a lambda graph with adjancy matrix and stuff.
nb_nodes = 10
nb_edges = 20

def random_adj(nb_nodes, nb_edges):
    # nodes
    nodes = np.arange(nb_nodes)

    # roughly nb_edges edges
    edges = np.array([(i, ((((i + np.random.randint(nb_nodes - 1))  % nb_nodes) + 1 ) % nb_nodes ))
                      for i in [np.random.randint(nb_nodes) for i in range(nb_edges)]])

    # Adding self loop.
    edges = np.concatenate((edges, np.array([(i, i) for i in nodes])))


    # adjacent matrix
    A = np.zeros((nb_nodes, nb_nodes))
    A[edges[:, 0], edges[:, 1]] = 1.
    A[edges[:, 1], edges[:, 0]] = 1.

    # Degree matrix
    D = A.sum(axis=1)
    
    return A, D

A, D = random_adj(nb_nodes, nb_edges)

In [5]:
print "Adjency matrix: \n{} \n nb edges: {} \n Degree: {} ".format(A, A.sum(), D)


Adjency matrix: 
[[ 1.  0.  0.  1.  1.  0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  1.  1.  0.  1.  0.  0.  1.]
 [ 1.  0.  0.  1.  1.  0.  0.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  1.  1.  1.  0.  0.]
 [ 0.  0.  1.  1.  0.  1.  1.  1.  1.  1.]
 [ 0.  0.  0.  0.  1.  1.  1.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.]
 [ 1.  0.  0.  1.  0.  0.  1.  0.  0.  1.]] 
 nb edges: 40.0 
 Degree: [ 4.  2.  2.  5.  4.  4.  7.  5.  3.  4.] 


In [6]:
# Get the Normalized matrix :D^(-1/2)AD^(-1/2)
np.set_printoptions(precision=2)
D_inv = np.diag(1./np.sqrt(D))
norm_transform = D_inv.dot(A).dot(D_inv)
print norm_transform
# So it's not only an average, it's weighted by something (need to investigate why).
# From what I can tell it's kind of a random walk throught the graph. 
# (i.e. the weights on edges connecting "popular" nodes are smaller)

[[ 0.25  0.    0.    0.22  0.25  0.    0.    0.    0.    0.25]
 [ 0.    0.5   0.    0.    0.    0.35  0.    0.    0.    0.  ]
 [ 0.    0.    0.5   0.    0.    0.    0.27  0.    0.    0.  ]
 [ 0.22  0.    0.    0.2   0.22  0.    0.17  0.    0.    0.22]
 [ 0.25  0.    0.    0.22  0.25  0.    0.    0.22  0.    0.  ]
 [ 0.    0.35  0.    0.    0.    0.25  0.19  0.22  0.    0.  ]
 [ 0.    0.    0.27  0.17  0.    0.19  0.14  0.17  0.22  0.19]
 [ 0.    0.    0.    0.    0.22  0.22  0.17  0.2   0.26  0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.22  0.26  0.33  0.  ]
 [ 0.25  0.    0.    0.22  0.    0.    0.19  0.    0.    0.25]]


In [70]:
# Create a module for the CGN:
class CGN(nn.Module):

    def __init__(self, nb_nodes, input_dim, channels, adj, out_dim=None, 
                ):
        super(CGN, self).__init__()

        self.my_layers = []
        self.out_dim = out_dim
        
        self.adj = adj # save our adjacence matrix
        self.edges = torch.LongTensor(np.array(np.where(adj))) # The list of edges
        self.D_norm = self.compute_D_norm(adj) # The normalization transformation
        flat_D_norm = self.D_norm.flatten()[np.where(self.D_norm.flatten())] # get the value
        flat_D_norm = torch.FloatTensor(flat_D_norm)
        
        # Constructing a sparse matrix
        self.sparse_D_norm = torch.sparse.FloatTensor(self.edges, flat_D_norm, torch.Size([nb_nodes,nb_nodes])).to_dense()
        self.sparse_D_norm = Variable(self.sparse_D_norm).cuda()
        
        dims = [input_dim] + channels
        
        layers = []
        for c_in, c_out in zip(dims[:-1], dims[1:]):
            layer = nn.Conv1d(c_in, c_out, 1, bias=False)
            layers.append(layer)
        self.my_layers = nn.ModuleList(layers)
        
        # If we have only one target per graph, we have a linear layer.
        if out_dim is not None:
            self.last_layer = nn.Linear(nb_nodes * channels[-1], out_dim)
        
    def compute_D_norm(self, adj):
        
        D = adj.sum(0)
        D_inv = np.diag(1./np.sqrt(D))
        norm_transform = D_inv.dot(A).dot(D_inv)
        return norm_transform # The normalizing matrix.
    
    
    def forward(self, x):
        
        nb_examples, nb_nodes, nb_channels = x.size() 

        def batch_mul(x, D):
            nb_examples, nb_channels, nb_nodes = x.size()
            x = x.view(-1, nb_nodes)
            x = x.mm(D)
            x = x.view(nb_examples, nb_channels, nb_nodes)
            
            return x

        x = x.permute(0, 2, 1).contiguous()# from ex, node, ch, -> ex, ch, node 
        
        # Do graph convolution for all 
        for layer in self.my_layers:
            x=x.cuda()
            
            x = batch_mul(x, self.sparse_D_norm)
            x = F.tanh(layer(x)) # or relu, sigmoid...
            
        if self.out_dim is not None:
            x = self.last_layer(x.view(nb_examples, -1))
        
        return x

In [76]:
# Generate some random data:
nb_examples = 1000 # examples
nb_out = 1 # the umber of output (for classification)
nb_nodes = 5000
nb_edges = 5000
A, D =  random_adj(nb_nodes, nb_edges)

# Generate random stuff.
inputs = Variable(torch.randn((nb_examples, nb_nodes, 1)), requires_grad=False)
#targets = Variable(torch.randn((nb_examples, nb_out)), requires_grad=False)
targets = Variable(torch.sum(inputs.data, dim=1), requires_grad=False).squeeze() # try to predict the sum.


In [77]:
# Create our model.
cgn = CGN(nb_nodes, 1, [16] * 3, A, nb_out)

print "Our model:"
print cgn


Our model:
CGN (
  (my_layers): ModuleList (
    (0): Conv1d(1, 16, kernel_size=(1,), stride=(1,), bias=False)
    (1): Conv1d(16, 16, kernel_size=(1,), stride=(1,), bias=False)
    (2): Conv1d(16, 16, kernel_size=(1,), stride=(1,), bias=False)
  )
  (last_layer): Linear (80000 -> 1)
)


In [78]:
# put everything on gpu
cgn.cuda()
inputs.cuda()
targets.cuda()
print "Done putting everything on cuda."

Done putting everything on cuda.


In [79]:
# Train the cgn
learning_rate = 1e-4
criterion = torch.nn.MSELoss(size_average=True)
optimizer = torch.optim.SGD(cgn.parameters(), lr=learning_rate, momentum=0.9)

epoch = 500
for t in range(epoch):
    
    targets.cuda()
    cgn.cuda()
    inputs.cuda()
    
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = cgn(inputs[:500]).cuda()

    # Compute and print loss
    loss = criterion(y_pred, targets[:500].cuda())
    
    if t % (epoch/10) == 0:
        print(t, loss.data[0])

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print "Done!"

(0, 5447.0390625)
(50, 38.55936050415039)
(100, 0.23164597153663635)
(150, 0.0013994588516652584)


KeyboardInterrupt: 

In [80]:
# Check the results, and compare them
outputs = cgn(inputs)
print outputs[-10:], targets[-10:]
# Good enough for me.

Variable containing:
-55.4393
 58.3262
 -2.5749
 13.2410
-66.9487
 29.8692
 53.4250
 25.9610
 -3.2124
 56.9583
[torch.cuda.FloatTensor of size 10x1 (GPU 0)]
 Variable containing:
-141.5787
 -19.4537
  19.9199
  19.1533
   0.9837
  75.3365
  71.9767
  11.3443
  32.2488
  97.9933
[torch.FloatTensor of size 10]

